# Welcome to _the Coffee Case_ 

Today we are going to simulate and analyze costs for an imaginary coffee company, _Coffee Express_ located in [Rochester, MN](https://www.google.com/maps/place/Rochester,+MN,+USA/@43.996149,-92.6212554,11z/data=!3m1!4b1!4m5!3m4!1s0x87f75f4ba35f2c4b:0xf0951c175661c63d!8m2!3d44.0121221!4d-92.4801989).

Basic information:

- Owner: Vic Bernard
- Two locations
    1. Located by [Century High School](https://www.google.com/maps/place/Century+High+School/@44.0478938,-92.4459926,14z/data=!4m5!3m4!1s0x87f9e05c5188bbc9:0x7238525853d5f344!8m2!3d44.0500221!4d-92.4250403)
    2. Located "Downtown" by the [Mayo Clinic](https://www.google.com/maps/place/Mayo+Clinic/@44.0222436,-92.4675056,17z/data=!4m5!3m4!1s0x87f75f7b3595d707:0x2cf7d7fd5e643d1b!8m2!3d44.0226255!4d-92.4668833)
    
 Vic wants to understand the cost structure of his business. He's been keeping all of his reciepts, but he doesn't have a good sense of which activities really drive costs.

# This file simulates cost data for Coffee Express. Then estimates cost structure.
### Here are the activities that we measure weekly:
- Cappuchino
- card transactions
- orders
- 2 locations
### To run this notebook on your machine you need Anaconda
[Download Anaconda for Python 3.6 here](https://www.anaconda.com/download/)

In [2]:
# the main data management packages
import numpy as np
import pandas as pd
# for basic statistical analysis
import statsmodels.api as sm
# this just makes the notebook full screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# set up the generator
np.random.seed(10)
size=104 # 52 weeks

In [4]:
# feature setup location 1

loc1 = np.ones(size) # indicates loc 1 with a 1

totalCups_l1 = np.random.choice(a=range(3000,5000), size=size) # location 1 is bigger

ordersFactor_l1 = np.random.choice(a=range(50,100), size=size) # but more people order together
ordersFactor_l1 = np.divide(ordersFactor_l1, 100) # make it a factor (this is a stupid way to do this... but whatever)

orders_l1 = totalCups_l1 * ordersFactor_l1

capFactor_l1 = np.random.choice(a=range(0,25), size=size) # less capps 
capFactor_l1 = np.divide(capFactor_l1, 100) # make it a factor

capps_l1 = totalCups_l1 * capFactor_l1

cardFactor_l1 = np.random.choice(a=range(0,25), size=size) # less cards
cardFactor_l1 = np.divide(cardFactor_l1, 100) # make it a factor

cards_l1 = orders_l1 * cardFactor_l1

togoFactor_l1 = np.random.choice(a=range(60,90), size=size) # less 
togoFactor_l1 = np.divide(togoFactor_l1, 100) # make it a factor


# feature setup location 2

loc2 = np.zeros(size)

totalCups_l2 = np.random.choice(a=range(500,1500), size=size) # smaller location

ordersFactor_l2 = np.random.choice(a=range(0,75), size=size) # fewer group orders
ordersFactor_l2 = np.divide(ordersFactor_l2, 100) # make it a factor

orders_l2 = totalCups_l2 * ordersFactor_l2

capFactor_l2 = np.random.choice(a=range(50,100), size=size) # more capps 
capFactor_l2 = np.divide(capFactor_l2, 100) # make it a factor

capps_l2 = totalCups_l2 * capFactor_l2

cardFactor_l2 = np.random.choice(a=range(75,100), size=size) # more cards
cardFactor_l2 = np.divide(cardFactor_l2, 100) # make it a factor

cards_l2 = orders_l2 * cardFactor_l2

togoFactor_l2 = np.random.choice(a=range(60,90), size=size) # less info 
togoFactor_l2 = np.divide(togoFactor_l1, 100) # make it a factor


In [4]:
myDict1 =  {"cups":totalCups_l1, "capps":capps_l1, "orders":orders_l1, "cards":cards_l1, "locationOne":loc1 }
myDict2 =  {"cups":totalCups_l2, "capps":capps_l2, "orders":orders_l2, "cards":cards_l2, "locationOne":loc2 }

In [5]:
location1 = pd.DataFrame(myDict1, columns=["cups","capps", "orders", "cards", "locationOne"])
location2 = pd.DataFrame(myDict2, columns=["cups","capps", "orders", "cards", "locationOne"])
CoffeeData = location1.append(location2)
CoffeeData = CoffeeData.round()

In [6]:
CoffeeData.head()

,cups,capps,orders,cards,locationOne
0,4289,386.0,4203.0,420.0,1.0
1,4149,290.0,2572.0,103.0,1.0
2,3527,776.0,2751.0,605.0,1.0
3,4344,0.0,3562.0,677.0,1.0
4,4393,132.0,4349.0,913.0,1.0


In [9]:
# generate cost equation
CostData = 500 + (CoffeeData.cups-CoffeeData.capps)*.20 + CoffeeData.capps*.70 + CoffeeData.orders * .5 + CoffeeData.cards * 1.24
# add error
efactor = np.random.randn(len(CostData))
efactor = efactor/5
CostData = CostData + efactor*CostData
CostData = CostData.round(2)

In [5]:
# lets inspect the cost data array:
CostData

NameError: name 'CostData' is not defined










# Let's figure out what is driving cost at Coffee Express!!

### What is  Vic's Hypothesis:

He predicts that:

$Cost = \alpha + \beta_0 Loc1 + \beta_1 Cups $

With:
 - Fixed costs: 
     - Equal USD 500 so 
         - $\alpha > 0$
     - Same at both locations so 
         - $\beta_0 = 0$
 - Variable costs:
     - Number of cups is the activity cost driver
         - $\beta_1 > 0$

In [11]:
# set up the data for the regressions
myDictVicModel =  {"cups":CoffeeData.cups, "locationOne":CoffeeData.locationOne }
vicData = pd.DataFrame(myDictVicModel)

In [12]:
cost = CostData
Xvic = vicData
Xvic = sm.add_constant(Xvic)
Xall = CoffeeData
Xall = sm.add_constant(Xall)

/Users/arthurmorris/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [13]:
model = sm.OLS(cost, Xvic).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.689
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     227.3
Date:                Tue, 24 Mar 2020   Prob (F-statistic):           9.39e-53
Time:                        17:18:35   Log-Likelihood:                -1651.5
No. Observations:                 208   AIC:                             3309.
Df Residuals:                     205   BIC:                             3319.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         816.8479    123.186      6.631      0.000     573.974    1059.722
cups            0.8451      0.107      7.898      0.000       0.634       1.056
locationOne  -657.4897    334.881     -1.963      0.051   -1317.742       2.762
==============================================================================
Omnibus:                        4.457   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.108   Jarque-Bera (JB):                5.509
Skew:                          -0.106   Prob(JB):                       0.0636
Kurtosis:                       3.769   Cond. No.                     2.13e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Is Vic correct? How do we know?

## First are our estimates consistant with his hypotheses?
- The constant in greater than zero. But it seems high.
- Cups are positive.
- Location One is not equal to zero.
## Could these just be due to randomness?
### Are these estimates significant _relative to the noise in the sample_?
- The __T-Statistic__ is a measure of this concept which we can use to calculate the probability that each parameter is only a product of noise.

$ t = \frac{\hat{\beta}-\beta_H}{SE(\hat{\beta})} $

- Where $\beta_H$ is the hypothesis (Regression output tests $\beta_H = 0$)
- $\hat{\beta}$ is the estimated parameter
- $SE(\hat{\beta})$ is the Standard Error of the estimate (a measure of the noise in the estimate)
### So, for the constant:

$ t=\frac{816.85 - 0}{123.19}=6.63 $

### How likely is it that our estimate is just due to randomness?
- Use the t-stat and the degrees of freedom to get the probability from the t-table (note we are looking at a one-tailed test)
### What if our hypothesis is $\alpha = 500$?:

$ t=\frac{816.85 - 500}{123.19}=2.57 $

# Conclusion:
1. Our model doesn't work like we expect. And not in a good way. 
2. The estimate for Location 1 is BONKERS.

### We need to improve the model

# New Hypotheses:
- Remember a complete model will
    - Explain as much variation in cost as possible given the data
    - correctly separate fixed and variable costs 
        ($\alpha = 500\;\; \&\;\; \beta_{Loc1}=0$)

# H1: Credit Card use at the two locations drives the difference

$Cost = \alpha + \beta_0 Cards + \beta_1 Location1 $
- $\alpha = 500$
- $\beta_0 Cards > 0$
- $\beta_1 Location1 = 0$

In [ ]:
# make h1 data
h1Data = CoffeeData.drop(['cups', 'capps', 'orders'], axis=1)
Xh1 = sm.add_constant(h1Data)
Xh1.head()


In [ ]:
model = sm.OLS(cost, Xh1).fit()
model.summary()

# H2: Cups, Cards, and Coffee Type
$$Cost = \alpha + \beta_0 Cards + \beta_1 Cups + \beta_2 Capps + \beta_3 Location1 $$


In [14]:
h2Data = CoffeeData.drop(['orders'], axis=1)
Xh2 = sm.add_constant(h2Data)
Xh2.head()


,const,cups,capps,cards,locationOne
0,1.0,4289,386.0,420.0,1.0
1,1.0,4149,290.0,103.0,1.0
2,1.0,3527,776.0,605.0,1.0
3,1.0,4344,0.0,677.0,1.0
4,1.0,4393,132.0,913.0,1.0


In [15]:
model = sm.OLS(cost, Xh2).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.785
Method:                 Least Squares   F-statistic:                     190.1
Date:                Tue, 24 Mar 2020   Prob (F-statistic):           1.87e-67
Time:                        17:24:31   Log-Likelihood:                -1611.1
No. Observations:                 208   AIC:                             3232.
Df Residuals:                     203   BIC:                             3249.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         181.0177    139.357      1.299      0.195     -93.756     455.791
cups            0.5138      0.096      5.325      0.000       0.324       0.704
capps           0.6675      0.161      4.147      0.000       0.350       0.985
cards           1.5854      0.172      9.205      0.000       1.246       1.925
locationOne   438.1852    313.178      1.399      0.163    -179.315    1055.685
==============================================================================
Omnibus:                        6.607   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.037   Jarque-Bera (JB):               10.744
Skew:                          -0.042   Prob(JB):                      0.00464
Kurtosis:                       4.110   Cond. No.                     2.38e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.38e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# H3 activity volumes are different at the two locations:
- we'll just compare means at the two locations

In [ ]:
location1.mean()

In [ ]:
location2.mean()

1. Lots more Capps at location 2
    - nearly twice as many capps in about 30% of the total amount of cups
2. Relative card useage is also higher at location 2

# H4: Add Orders
$$Cost = \alpha + \beta_0 Cups + \beta_2 Capps + \beta_3 Orders + \beta_4 Cards + \beta_5 Loc1 $$


In [16]:
model = sm.OLS(cost, Xall).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                     202.6
Date:                Tue, 24 Mar 2020   Prob (F-statistic):           1.16e-76
Time:                        17:25:36   Log-Likelihood:                -1586.4
No. Observations:                 208   AIC:                             3185.
Df Residuals:                     202   BIC:                             3205.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         501.4168    131.497      3.813      0.000     242.134     760.699
cups           -0.0435      0.115     -0.380      0.705      -0.269       0.182
capps           0.8433      0.145      5.805      0.000       0.557       1.130
orders          0.6715      0.091      7.355      0.000       0.491       0.852
cards           1.1122      0.166      6.688      0.000       0.784       1.440
locationOne   349.4008    279.089      1.252      0.212    -200.900     899.702
==============================================================================
Omnibus:                        9.455   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.009   Jarque-Bera (JB):               18.665
Skew:                           0.096   Prob(JB):                     8.85e-05
Kurtosis:                       4.455   Cond. No.                     2.96e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.96e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- So cups isn't significant at all!
- But I put it in the original data generating process... 
- The problem is that we don't have much independent variation! We need the variables to be independant of one another.

# What if we drop cups?

In [17]:
# make a NO-CUPS version:
noCupData = CoffeeData.drop(['cups'], axis=1)
Xnocup = sm.add_constant(noCupData)


In [18]:
#Xnocup

In [19]:
model = sm.OLS(cost, Xnocup).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                     254.3
Date:                Tue, 24 Mar 2020   Prob (F-statistic):           7.35e-78
Time:                        17:28:17   Log-Likelihood:                -1586.5
No. Observations:                 208   AIC:                             3183.
Df Residuals:                     203   BIC:                             3200.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         479.6462    118.091      4.062      0.000     246.804     712.489
capps           0.8243      0.136      6.057      0.000       0.556       1.093
orders          0.6486      0.068      9.492      0.000       0.514       0.783
cards           1.1169      0.165      6.749      0.000       0.791       1.443
locationOne   276.2031    201.401      1.371      0.172    -120.903     673.309
==============================================================================
Omnibus:                        9.494   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.009   Jarque-Bera (JB):               19.187
Skew:                           0.070   Prob(JB):                     6.82e-05
Kurtosis:                       4.481   Cond. No.                     1.36e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.36e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# SAVE THE COFFEE DATA

In [ ]:
CoffeeFile = CoffeeData
CoffeeFile['cost']=CostData

In [ ]:
CoffeeFile.to_csv('CoffeeFile.csv')